In [2]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
# print(OPENAI_API_KEY, " : ",  PINECONE_API_KEY)

In [3]:
from pinecone import Pinecone, ServerlessSpec

# Pinecone 클라이언트를 초기화합니다.
# PINECONE_API_KEY는 환경 변수에서 가져온 API 키입니다.
pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
# 1. 필요한 라이브러리 설치 (터미널에서 실행)
# pip install langchain pypdf

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
# Pinecone 인덱스를 생성
index_name ="lowbirth"
# 인덱스 이름은 "wiki"이고, 차원은 1536, 메트릭은 코사인 유사도를 사용
# 인덱스는 AWS의 us-east-1 리전에서 서버리스 사양으로 생성
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원, openapi embeding model을 사용함. 정확하게 일치
    metric="cosine",  # 모델 메트릭, openapi embeding model 에서 사용하는 것 확인
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "lowbirth",
    "metric": "cosine",
    "host": "lowbirth-j7upu7f.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [10]:
# [단계 1] PDF 로드 (메모리에 올리기)
# 파일 경로를 넣으면 텍스트를 추출해서 Document 객체로 만듭니다.
loader = PyPDFLoader("asiabrief_3-26.pdf")
data = loader.load()

In [11]:

from langchain_text_splitters import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(chunk_size=400, 
                                          chunk_overlap=30, 
                                          length_function =len,
                                          separators=['\n\n','\n',' ',''])

In [12]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small', api_key=OPENAI_API_KEY)

In [13]:
from tqdm.auto import tqdm
from uuid import uuid4
import time
from pinecone import Pinecone

# 1. Pinecone 클라이언트 초기화
pc = Pinecone(api_key=PINECONE_API_KEY)

# 2. 인덱스 이름 설정 (Pinecone 대시보드에서 생성한 인덱스 이름을 정확히 입력하세요)
index_name = "lowbirth" # <-- 실제 생성하신 인덱스 이름으로 변경해주세요!

# 3. 인덱스 객체 정의 (여기서 'index' 변수가 생성됩니다)
index = pc.Index(index_name)

# --- 이제 기존의 upsert 코드가 작동합니다 ---
# 업로드 설정
BATCH_SIZE = 100  # 100개씩 Vector DB에 저장
NAMESPACE = "lowbirth_1"
texts = []        # chunk 텍스트 모음
metadatas = []    # 메타데이터 모음

# 문서 하나씩 처리하기
# data : Wikipedia 문서 목록
for doc in tqdm(data):
    # full_text : 하나의 위키 문서 전체 내용
    full_text = doc.page_content
    source = doc.metadata.get("source", "unknown")
    page_num = doc.metadata.get("page", 0)

    # 문서 정보(메타데이터) 준비
    chunk_metadata = {
        "source": source,
        "page": page_num,
        # 위키피디아 예제용 키(wiki_id, title 등) 대신 PDF에 맞는 키를 사용하거나 직접 지정하세요.
        "title": "asiabrief_3-26" 
    }

    # 문서를 chunk로 나누기(너무 긴 문서는 LLM이 처리 못함, 작은 조각(chunk)으로 나눔)
    chunks = splitter.split_text(full_text)

    for idx, chunk in enumerate(chunks):
        # texts / metadatas에 임시 저장
        texts.append(chunk)
        metadatas.append({
            "chunk_id": idx,   # chunk_id : 같은 문서 내 몇 번째 조각인지
            # "chunk_text": chunk[:100], 
            "chunk_text": chunk, 
            **chunk_metadata
        })
        # 100개 모이면 Vector DB에 저장
        if len(texts) == BATCH_SIZE:
            ids = [str(uuid4()) for _ in range(BATCH_SIZE)]
        
            # 임베딩 -> Pinecone 업로드
            embeddings = embedding.embed_documents(texts)

            index.upsert(
                vectors=zip(ids, embeddings, metadatas),
                namespace=NAMESPACE
            )
            # 초기화 후 다음 배치로 
            texts = []
            metadatas = []
            time.sleep(1)

# 마지막 남은 chunk 업로드 누락
if texts:
    ids = [str(uuid4()) for _ in range(len(texts))]
    embeddings = embedding.embed_documents(texts)
    index.upsert(
        vectors=zip(ids, embeddings, metadatas),
        namespace=NAMESPACE
    )

  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from langchain_openai import OpenAIEmbeddings
# embedding 객체 생성
embedding = OpenAIEmbeddings(
    model = "text-embedding-3-small",
    # api_key = OPENAI_API_KEY # 생략가능
)
embedding